In [286]:
import torch
%pylab
%matplotlib inline

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [292]:
def gen_ds(M: int, k1: float, k2: float, b: float, batch_size=10) -> tuple:
    X = torch.rand(M, 2) * 100
    Y = k1 * X[:,0] + k2 * X[:,1] + b + torch.randn(M)
    X_batches = torch.stack(torch.split(X, batch_size))
    Y_batches = torch.stack(torch.split(X, batch_size))
    return X_batches, Y_batches, (k1, k2, b)

def shuffle_batches(X: torch.Tensor, Y: torch.Tensor) -> tuple:
    X = X[torch.randperm(X.shape[0])].view(X.shape)
    Y = Y[torch.randperm(Y.shape[0])].view(Y.shape)
    return X, Y

def clip_grad(grad: torch.Tensor, max_grad_len: float) -> tuple:
    grad_len = torch.norm(grad)
    if grad_len < max_grad_len:
        return grad, grad_len
    return (grad / grad_len) * max_grad_len, grad_len

def approx_k1k2b(k1: float, k2: float, b: float, M: int, epochs=1, alpha=.01, step=1, max_grad_len=1.) -> tuple:
    X_batches, Y_batches, _ = gen_ds(M, k1, k2, b)
    k1k2b = torch.tensor([1., 1., 0.], dtype=torch.float, requires_grad=True)
    for epoch in range(epochs):
        X_batches, Y_batches = shuffle_batches(X_batches, Y_batches)
        for x_batch, y_batch in zip(X_batches, Y_batches):
            yy = k1k2b[0] * x_batch[:,0] + k1k2b[1] * x_batch[:,1] + k1k2b[2]
            print(yy)
            print('\n\n\n', y_batch)
            mse = ((yy - y_batch)**2).mean()
            mse.backward()
            clipped_grad, _ = clip_grad(k1k2b.grad, max_grad_len)
            k1k2kb = torch.tensor(k1k2b - clipped_grad * alpha * step)
    return k1k2b[0].item(), k1k2b[1].item(), k1k2b[2].item()

In [293]:
approx_k1k2b(1., 2., 3., 1000)

tensor([149.5231, 140.8339, 112.9755,  96.2219, 112.2466,  85.6545, 125.3837,
         60.0117, 145.3310, 115.5002], grad_fn=<AddBackward0>)



 tensor([[42.8536, 95.1193],
        [81.1504, 21.5509],
        [24.2941, 56.3623],
        [ 5.1458, 81.7853],
        [35.2678, 20.4255],
        [69.5537, 86.3680],
        [ 7.0211, 66.8708],
        [79.9507, 98.4815],
        [12.5995, 48.7345],
        [62.4040,  1.9800]])


RuntimeError: The size of tensor a (10) must match the size of tensor b (2) at non-singleton dimension 1